## Group 7
## Big Data Group Project

### Contributors:
- 29233798: Joel Shien Yee Chin [Joel]
- 18435688: Tim O'Doherty [Tim]
- 22606127: Sean Whitehead [Sean]
- 29650437: Lin Bai [Lin]


### Phase 2: Cleansing and Reshaping

This notebook is used for co-developing of codes required to wrangle the home credit risk dataset.

Please use this notebook as the live version and download local copies as you require.

If you have picked up a specific section, please comment your name to the code cell so that we can work on other sections.

Helpful link for Dataframes in Scala:
- https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-scala.html#work-with-dataframes
- https://spark.apache.org/docs/1.5.0/api/java/overview-summary.html
- https://spark.apache.org/docs/2.2.0/sql-programming-guide.html
- https://www.tutorialspoint.com/scala/
- http://allaboutscala.com/big-data/spark/#dataframe-select-columns

Insert libraries into the cell below:

In [4]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql._
import org.apache.hadoop.io.LongWritable
import org.apache.hadoop.io.Text
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.mapreduce.lib.input.TextInputFormat
import org.apache.spark.sql.functions.{sum, col}
import org.apache.spark.sql.functions.{col, count, when}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.IntegerType

import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql._
import org.apache.hadoop.io.LongWritable
import org.apache.hadoop.io.Text
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.mapreduce.lib.input.TextInputFormat
import org.apache.spark.sql.functions.{sum, col}
import org.apache.spark.sql.functions.{col, count, when}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.IntegerType


Insert code for data loading below:

Download zip from GDrive and extract to location where you are running notebook for below to run

In [3]:
//read in csv [Sean]
val creditRiskdf1 = spark.read.option("header","true").csv("Home credit default risk after feature selection/m_train_small.csv")

creditRiskdf1: org.apache.spark.sql.DataFrame = [EXT_SOURCE_1: string, EXT_SOURCE_2: string ... 359 more fields]


In [4]:
//take a look at features in dataset (...361 of them) [Sean]
creditRiskdf1.printSchema()

root
 |-- EXT_SOURCE_1: string (nullable = true)
 |-- EXT_SOURCE_2: string (nullable = true)
 |-- EXT_SOURCE_3: string (nullable = true)
 |-- client_installments_AMT_PAYMENT_min_sum: string (nullable = true)
 |-- DAYS_BIRTH: string (nullable = true)
 |-- AMT_CREDIT: string (nullable = true)
 |-- AMT_ANNUITY: string (nullable = true)
 |-- DAYS_EMPLOYED: string (nullable = true)
 |-- bureau_DAYS_CREDIT_ENDDATE_max: string (nullable = true)
 |-- bureau_DAYS_CREDIT_max: string (nullable = true)
 |-- DAYS_ID_PUBLISH: string (nullable = true)
 |-- bureau_AMT_CREDIT_MAX_OVERDUE_mean: string (nullable = true)
 |-- bureau_DAYS_ENDDATE_FACT_max: string (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_DEBT_mean: string (nullable = true)
 |-- OWN_CAR_AGE: string (nullable = true)
 |-- previous_loans_CNT_PAYMENT_mean: string (nullable = true)
 |-- client_cash_CNT_INSTALMENT_FUTURE_min_max: string (nullable = true)
 |-- previous_loans_SELLERPLACE_AREA_max: string (nullable = true)
 |-- DAYS_LAST_PHONE_C

 |-- client_installments_NUM_INSTALMENT_NUMBER_mean_min: string (nullable = true)
 |-- client_bureau_balance_STATUS_C_count_mean: string (nullable = true)
 |-- client_installments_NUM_INSTALMENT_NUMBER_sum_min: string (nullable = true)
 |-- client_credit_CNT_DRAWINGS_CURRENT_max_sum: string (nullable = true)
 |-- client_credit_AMT_PAYMENT_CURRENT_sum_sum: string (nullable = true)
 |-- previous_loans_NAME_PORTFOLIO_XNA_count_norm: string (nullable = true)
 |-- previous_loans_NAME_CLIENT_TYPE_Repeater_count_norm: string (nullable = true)
 |-- previous_loans_NAME_SELLER_INDUSTRY_Connectivity_count_norm: string (nullable = true)
 |-- bureau_CREDIT_TYPE_Credit card_count: string (nullable = true)
 |-- client_credit_AMT_CREDIT_LIMIT_ACTUAL_sum_sum: string (nullable = true)
 |-- previous_loans_NAME_CONTRACT_STATUS_Canceled_count_norm: string (nullable = true)
 |-- client_bureau_balance_STATUS_0_count_norm_max: string (nullable = true)
 |-- OCCUPATION_TYPE_Drivers: string (nullable = true)
 |-

 |-- previous_loans_PRODUCT_COMBINATION_Card Street_count_norm: string (nullable = true)
 |-- client_installments_NUM_INSTALMENT_NUMBER_max_min: string (nullable = true)
 |-- client_credit_AMT_PAYMENT_CURRENT_min_sum: string (nullable = true)
 |-- previous_loans_PRODUCT_COMBINATION_POS industry without interest_count_norm: string (nullable = true)
 |-- previous_loans_CHANNEL_TYPE_Regional / Local_count: string (nullable = true)
 |-- ORGANIZATION_TYPE_Transport: type 3: string (nullable = true)
 |-- client_cash_NAME_CONTRACT_STATUS_Returned to the store_count_norm_max: string (nullable = true)
 |-- ORGANIZATION_TYPE_School: string (nullable = true)
 |-- previous_loans_NAME_TYPE_SUITE_Other_B_count_norm: string (nullable = true)
 |-- bureau_CNT_CREDIT_PROLONG_mean: string (nullable = true)
 |-- client_credit_AMT_DRAWINGS_CURRENT_max_max: string (nullable = true)
 |-- client_bureau_balance_MONTHS_BALANCE_max_sum: string (nullable = true)
 |-- client_cash_NAME_CONTRACT_STATUS_Active_count_

In [7]:
//string array of features to pass through EDA functions [Sean]
val creditRiskFeatures = creditRiskdf1.columns

creditRiskFeatures: Array[String] = Array(EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3, client_installments_AMT_PAYMENT_min_sum, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_EMPLOYED, bureau_DAYS_CREDIT_ENDDATE_max, bureau_DAYS_CREDIT_max, DAYS_ID_PUBLISH, bureau_AMT_CREDIT_MAX_OVERDUE_mean, bureau_DAYS_ENDDATE_FACT_max, bureau_AMT_CREDIT_SUM_DEBT_mean, OWN_CAR_AGE, previous_loans_CNT_PAYMENT_mean, client_cash_CNT_INSTALMENT_FUTURE_min_max, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, bureau_AMT_CREDIT_SUM_max, bureau_AMT_CREDIT_SUM_mean, DAYS_REGISTRATION, client_installments_DAYS_INSTALMENT_max_max, previous_loans_AMT_DOWN_PAYMENT_max, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_UPDATE_max, bureau_AMT_CREDIT_SUM_sum, clie...

In [6]:
//ID col summary (min to max) & number of rows [Sean]
creditRiskdf1.describe(creditRiskFeatures(360)).show()

+-------+------------------+
|summary|        SK_ID_CURR|
+-------+------------------+
|  count|            307511|
|   mean|278180.51857657125|
| stddev| 102790.1753484249|
|    min|            100002|
|    max|            456255|
+-------+------------------+



## Split and allocate features for data cleaning task execution

In [8]:
//split and allocate features for down stream cleaning [Sean]
var creditRiskFeatures1 = creditRiskFeatures.slice(0,120)
var creditRiskFeatures2 = creditRiskFeatures.slice(121,240)
var creditRiskFeatures3 = creditRiskFeatures.slice(241,360)
//create partitioned DFs for downstream cleaning [Sean]
//unpack array of column headers as strings
var creditRiskPartition1 = creditRiskdf1.select(creditRiskFeatures1.head, creditRiskFeatures1.tail: _*)
var creditRiskPartition2 = creditRiskdf1.select(creditRiskFeatures2.head, creditRiskFeatures2.tail: _*)
var creditRiskPartition3 = creditRiskdf1.select(creditRiskFeatures3.head, creditRiskFeatures3.tail: _*)

creditRiskFeatures1: Array[String] = Array(EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3, client_installments_AMT_PAYMENT_min_sum, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_EMPLOYED, bureau_DAYS_CREDIT_ENDDATE_max, bureau_DAYS_CREDIT_max, DAYS_ID_PUBLISH, bureau_AMT_CREDIT_MAX_OVERDUE_mean, bureau_DAYS_ENDDATE_FACT_max, bureau_AMT_CREDIT_SUM_DEBT_mean, OWN_CAR_AGE, previous_loans_CNT_PAYMENT_mean, client_cash_CNT_INSTALMENT_FUTURE_min_max, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, bureau_AMT_CREDIT_SUM_max, bureau_AMT_CREDIT_SUM_mean, DAYS_REGISTRATION, client_installments_DAYS_INSTALMENT_max_max, previous_loans_AMT_DOWN_PAYMENT_max, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_UPDATE_max, bureau_AMT_CREDIT_SUM_sum, cli...

### As the full dataset is huge, the codes require significant processing time and therefore are not conducive for review by the teaching team.  From here on, the codes will be run using a subset of the main dataset for the purpose of demonstrating functionality. Note that the data cleansing was performed on the full dataset and yielded 3 cleansed dataset (see report). To reperform the full data cleansing, remove the cell block below.

In [8]:
//subset of main dataset with 10% of rows [Sean]
//SUBSET OF MAIN DATA USED TO SHOW HOW CODE WORKS & COMPUTE IN A MORE TIMELY MANNER
//FOR ACTUAL DATA PREPARATION REMOVE THIS LINE AND RE-RUN NOTEBOOK
val creditRiskdf = creditRiskdf1.sample(false, 0.1)
val creditRiskFeatures = creditRiskdf.columns

creditRiskdf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_1: string, EXT_SOURCE_2: string ... 359 more fields]
creditRiskFeatures: Array[String] = Array(EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3, client_installments_AMT_PAYMENT_min_sum, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_EMPLOYED, bureau_DAYS_CREDIT_ENDDATE_max, bureau_DAYS_CREDIT_max, DAYS_ID_PUBLISH, bureau_AMT_CREDIT_MAX_OVERDUE_mean, bureau_DAYS_ENDDATE_FACT_max, bureau_AMT_CREDIT_SUM_DEBT_mean, OWN_CAR_AGE, previous_loans_CNT_PAYMENT_mean, client_cash_CNT_INSTALMENT_FUTURE_min_max, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, bureau_AMT_CREDIT_SUM_max, bureau_AMT_CREDIT_SUM_mean, DAYS_REGISTRATION, client_installments_DAYS_INSTALMENT_max_max, previous_loans_AMT_DOWN_PAYMENT_max, CODE_G...

## flagging missing values:

In [ ]:
//create DF of features & count of nulls
//set threshold e.g if 20% of values = null, drop column
//**EXTENTION** Impute values to replace null, will have to review feature properties ***IF WE HAVE TIME***

//sum count of int for null values [Lin]

In [10]:
val df = creditRiskdf

df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_1: string, EXT_SOURCE_2: string ... 359 more fields]


In [11]:
df.count()

2019-05-26 15:48:41,108 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


res2: Long = 30283


Check the percentage of NotNull value of each column:

In [12]:
val df4=df.select(df.columns.map(c => (count(c) / count("*")).alias(c)): _*)

df4: org.apache.spark.sql.DataFrame = [EXT_SOURCE_1: double, EXT_SOURCE_2: double ... 359 more fields]


In [13]:
df4.show()

+-------------------+-----------------+------------------+---------------------------------------+----------+----------+------------------+-------------+------------------------------+----------------------+---------------+----------------------------------+----------------------------+-------------------------------+-------------------+-------------------------------+-----------------------------------------+-----------------------------------+----------------------+-------------------------+--------------------------+-----------------+-------------------------------------------+-----------------------------------+-------------+--------------------------+-----------------------------------+-----------------------------+-------------------------+-------------------------------------------+---------------------------+---------------------------------------------------+-----------------------+----------------------------------------+-------------------------------------------+-----------

|       EXT_SOURCE_1|     EXT_SOURCE_2|      EXT_SOURCE_3|client_installments_AMT_PAYMENT_min_sum|DAYS_BIRTH|AMT_CREDIT|       AMT_ANNUITY|DAYS_EMPLOYED|bureau_DAYS_CREDIT_ENDDATE_max|bureau_DAYS_CREDIT_max|DAYS_ID_PUBLISH|bureau_AMT_CREDIT_MAX_OVERDUE_mean|bureau_DAYS_ENDDATE_FACT_max|bureau_AMT_CREDIT_SUM_DEBT_mean|        OWN_CAR_AGE|previous_loans_CNT_PAYMENT_mean|client_cash_CNT_INSTALMENT_FUTURE_min_max|previous_loans_SELLERPLACE_AREA_max|DAYS_LAST_PHONE_CHANGE|bureau_AMT_CREDIT_SUM_max|bureau_AMT_CREDIT_SUM_mean|DAYS_REGISTRATION|client_installments_DAYS_INSTALMENT_max_max|previous_loans_AMT_DOWN_PAYMENT_max|CODE_GENDER_F|REGION_POPULATION_RELATIVE|client_cash_CNT_INSTALMENT_max_mean|bureau_DAYS_CREDIT_UPDATE_max|bureau_AMT_CREDIT_SUM_sum|client_credit_AMT_DRAWINGS_CURRENT_mean_max|YEARS_BEGINEXPLUATATION_AVG|client_installments_NUM_INSTALMENT_VERSION_mean_max|bureau_DAYS_CREDIT_mean|client_installments_AMT_PAYMENT_sum_mean|client_installments_DAYS_INSTALMENT_min_max|previous_lo

n_sum|previous_loans_HOUR_APPR_PROCESS_START_min|client_credit_AMT_INST_MIN_REGULARITY_max_min|client_cash_NAME_CONTRACT_STATUS_Signed_count_norm_max|OCCUPATION_TYPE_Laborers|client_cash_CNT_INSTALMENT_min_max|previous_loans_WEEKDAY_APPR_PROCESS_START_THURSDAY_count_norm|ORGANIZATION_TYPE_Construction|client_cash_NAME_CONTRACT_STATUS_Active_count_max|client_cash_SK_DPD_DEF_sum_min|previous_loans_NAME_CONTRACT_TYPE_Cash loans_count_norm|previous_loans_DAYS_FIRST_DRAWING_mean|client_credit_CNT_DRAWINGS_ATM_CURRENT_max_sum|previous_loans_PRODUCT_COMBINATION_Cash X-Sell: middle_count_norm|previous_loans_NAME_CASH_LOAN_PURPOSE_Other_count_norm|client_bureau_balance_STATUS_0_count_norm_sum|client_installments_NUM_INSTALMENT_VERSION_sum_min|previous_loans_CHANNEL_TYPE_Country-wide_count_norm|client_cash_NAME_CONTRACT_STATUS_Signed_count_sum|AMT_REQ_CREDIT_BUREAU_YEAR|bureau_CREDIT_TYPE_Microloan_count|NAME_INCOME_TYPE_State servant|client_credit_AMT_CREDIT_LIMIT_ACTUAL_sum_max|previous_loans_

|0.43747316976521483|0.997688472080045|0.8019350790872767|                     0.9487171020044249|       1.0|       1.0|0.9999009345177162|          1.0|            0.8492883796189281|    0.8564871379982169|            1.0|                0.6009642373608955|          0.7470858237294852|             0.8332397714889542|0.33966251692368654|             0.9464385959118977|                       0.9422118020011228|                  0.947726447181587|                   1.0|       0.8564871379982169|        0.8564871379982169|              1.0|                         0.9487171020044249|                 0.8914902750718224|          1.0|                       1.0|                 0.9422118020011228|           0.8564871379982169|       0.8564871379982169|                         0.2836244757784896|         0.5082719677706964|                                 0.9487171020044249|     0.8564871379982169|                      0.9487171020044249|                         0.9487171020044249|           

587|                           0.2836244757784896|                          0.29399332959085955|                                      0.947726447181587|                                     0.947726447181587|                            0.2836244757784896|                         0.29399332959085955|             1.0|                                  0.947726447181587|                      0.8564871379982169|                         0.2836244757784896|                                          0.2836244757784896|                     0.29399332959085955|                           0.947726447181587|                                           0.947726447181587|                         0.947726447181587|                                    0.947726447181587|                            1.0|                                                  0.947726447181587|            0.9464385959118977|                         0.947726447181587|                         1.0|                                1.0|   

Check the count of null value in each column:

In [14]:
//df.select(df.columns.map(c => sum(col(c).isNull.cast("int")).alias(c)): _*).show
val df3=df.select(df.columns.map(c => sum(col(c).isNull.cast("int")).alias(c)): _*)

df3: org.apache.spark.sql.DataFrame = [EXT_SOURCE_1: bigint, EXT_SOURCE_2: bigint ... 359 more fields]


In [15]:
df3.show()

+------------+------------+------------+---------------------------------------+----------+----------+-----------+-------------+------------------------------+----------------------+---------------+----------------------------------+----------------------------+-------------------------------+-----------+-------------------------------+-----------------------------------------+-----------------------------------+----------------------+-------------------------+--------------------------+-----------------+-------------------------------------------+-----------------------------------+-------------+--------------------------+-----------------------------------+-----------------------------+-------------------------+-------------------------------------------+---------------------------+---------------------------------------------------+-----------------------+----------------------------------------+-------------------------------------------+--------------------------------------------

|EXT_SOURCE_1|EXT_SOURCE_2|EXT_SOURCE_3|client_installments_AMT_PAYMENT_min_sum|DAYS_BIRTH|AMT_CREDIT|AMT_ANNUITY|DAYS_EMPLOYED|bureau_DAYS_CREDIT_ENDDATE_max|bureau_DAYS_CREDIT_max|DAYS_ID_PUBLISH|bureau_AMT_CREDIT_MAX_OVERDUE_mean|bureau_DAYS_ENDDATE_FACT_max|bureau_AMT_CREDIT_SUM_DEBT_mean|OWN_CAR_AGE|previous_loans_CNT_PAYMENT_mean|client_cash_CNT_INSTALMENT_FUTURE_min_max|previous_loans_SELLERPLACE_AREA_max|DAYS_LAST_PHONE_CHANGE|bureau_AMT_CREDIT_SUM_max|bureau_AMT_CREDIT_SUM_mean|DAYS_REGISTRATION|client_installments_DAYS_INSTALMENT_max_max|previous_loans_AMT_DOWN_PAYMENT_max|CODE_GENDER_F|REGION_POPULATION_RELATIVE|client_cash_CNT_INSTALMENT_max_mean|bureau_DAYS_CREDIT_UPDATE_max|bureau_AMT_CREDIT_SUM_sum|client_credit_AMT_DRAWINGS_CURRENT_mean_max|YEARS_BEGINEXPLUATATION_AVG|client_installments_NUM_INSTALMENT_VERSION_mean_max|bureau_DAYS_CREDIT_mean|client_installments_AMT_PAYMENT_sum_mean|client_installments_DAYS_INSTALMENT_min_max|previous_loans_NAME_CONTRACT_STATUS_Refused_

_max|OCCUPATION_TYPE_Laborers|client_cash_CNT_INSTALMENT_min_max|previous_loans_WEEKDAY_APPR_PROCESS_START_THURSDAY_count_norm|ORGANIZATION_TYPE_Construction|client_cash_NAME_CONTRACT_STATUS_Active_count_max|client_cash_SK_DPD_DEF_sum_min|previous_loans_NAME_CONTRACT_TYPE_Cash loans_count_norm|previous_loans_DAYS_FIRST_DRAWING_mean|client_credit_CNT_DRAWINGS_ATM_CURRENT_max_sum|previous_loans_PRODUCT_COMBINATION_Cash X-Sell: middle_count_norm|previous_loans_NAME_CASH_LOAN_PURPOSE_Other_count_norm|client_bureau_balance_STATUS_0_count_norm_sum|client_installments_NUM_INSTALMENT_VERSION_sum_min|previous_loans_CHANNEL_TYPE_Country-wide_count_norm|client_cash_NAME_CONTRACT_STATUS_Signed_count_sum|AMT_REQ_CREDIT_BUREAU_YEAR|bureau_CREDIT_TYPE_Microloan_count|NAME_INCOME_TYPE_State servant|client_credit_AMT_CREDIT_LIMIT_ACTUAL_sum_max|previous_loans_DAYS_TERMINATION_max|client_credit_AMT_PAYMENT_TOTAL_CURRENT_max_max|FLAG_WORK_PHONE|previous_loans_NAME_GOODS_CATEGORY_Photo / Cinema Equipment_

|       17035|          70|        5998|                                   1553|         0|         0|          3|            0|                          4564|                  4346|              0|                             12084|                        7659|                           5050|      19997|                           1622|                                     1750|                               1583|                     0|                     4346|                      4346|                0|                                       1553|                               3286|            0|                         0|                               1750|                         4346|                     4346|                                      21694|                      14891|                                               1553|                   4346|                                    1553|                                       1553|                                            

              0|                                               1583|                                    4346|                                      21694|                                                       21694|                                   21380|                                        1583|                                                        1583|                                      1583|                                                 1583|                              0|                                                               1583|                          1622|                                      1583|                           0|                                  0|                            21694|                                                   1748|                                             1583|                                                     1583|                                             1553|                                    21694|             

Or: Express the count of null value in each column in forms of array:

In [16]:
val col= df.columns
var dfArray=col.map(colmn=>df.select(lit(colmn).as("colName"),sum(when(df(colmn).isNull,1).otherwise(0)).as("missingValues")))
dfArray.tail.foldLeft(dfArray.head)((acc,itr)=>acc.union(itr)).show

+--------------------+-------------+
|             colName|missingValues|
+--------------------+-------------+
|        EXT_SOURCE_1|        17035|
|        EXT_SOURCE_2|           70|
|        EXT_SOURCE_3|         5998|
|client_installmen...|         1553|
|          DAYS_BIRTH|            0|
|          AMT_CREDIT|            0|
|         AMT_ANNUITY|            3|
|       DAYS_EMPLOYED|            0|
|bureau_DAYS_CREDI...|         4564|
|bureau_DAYS_CREDI...|         4346|
|     DAYS_ID_PUBLISH|            0|
|bureau_AMT_CREDIT...|        12084|
|bureau_DAYS_ENDDA...|         7659|
|bureau_AMT_CREDIT...|         5050|
|         OWN_CAR_AGE|        19997|
|previous_loans_CN...|         1622|
|client_cash_CNT_I...|         1750|
|previous_loans_SE...|         1583|
|DAYS_LAST_PHONE_C...|            0|
|bureau_AMT_CREDIT...|         4346|
+--------------------+-------------+
only showing top 20 rows



col: Array[String] = Array(EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3, client_installments_AMT_PAYMENT_min_sum, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_EMPLOYED, bureau_DAYS_CREDIT_ENDDATE_max, bureau_DAYS_CREDIT_max, DAYS_ID_PUBLISH, bureau_AMT_CREDIT_MAX_OVERDUE_mean, bureau_DAYS_ENDDATE_FACT_max, bureau_AMT_CREDIT_SUM_DEBT_mean, OWN_CAR_AGE, previous_loans_CNT_PAYMENT_mean, client_cash_CNT_INSTALMENT_FUTURE_min_max, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, bureau_AMT_CREDIT_SUM_max, bureau_AMT_CREDIT_SUM_mean, DAYS_REGISTRATION, client_installments_DAYS_INSTALMENT_max_max, previous_loans_AMT_DOWN_PAYMENT_max, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_UPDATE_max, bureau_AMT_CREDIT_SUM_sum, client_credit_AMT_D...

From the above outcome we can see that there are lots of columns with a big portion of null value.
But if we remove all rows containing 'null', there would be only 800+ rows and 300+columns left, far from enought for
the following steps.

Hence we set a threshold as 92253, which is 30% of the number of total rows. And set a pipeline to select the columns 
in which the null counts is less than 92253, in other words, less than 30% of the column is emply.

In [17]:
val null_thresh = 92253              

val to_keep = df.columns.filter(
    c => df.agg(
        sum(when(df(c).isNull, 1).otherwise(0)).alias(c)
    ).first().getLong(0) <= null_thresh
)

//df.select(to_keep.head, to_keep.tail: _*).show()

null_thresh: Int = 92253
to_keep: Array[String] = Array(EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3, client_installments_AMT_PAYMENT_min_sum, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_EMPLOYED, bureau_DAYS_CREDIT_ENDDATE_max, bureau_DAYS_CREDIT_max, DAYS_ID_PUBLISH, bureau_AMT_CREDIT_MAX_OVERDUE_mean, bureau_DAYS_ENDDATE_FACT_max, bureau_AMT_CREDIT_SUM_DEBT_mean, OWN_CAR_AGE, previous_loans_CNT_PAYMENT_mean, client_cash_CNT_INSTALMENT_FUTURE_min_max, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, bureau_AMT_CREDIT_SUM_max, bureau_AMT_CREDIT_SUM_mean, DAYS_REGISTRATION, client_installments_DAYS_INSTALMENT_max_max, previous_loans_AMT_DOWN_PAYMENT_max, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_UPDATE_max, bureau_AMT_CREDIT...

In [18]:
val df5=df.select(to_keep.head, to_keep.tail: _*)

df5: org.apache.spark.sql.DataFrame = [EXT_SOURCE_1: string, EXT_SOURCE_2: string ... 359 more fields]


In [19]:
df5.count()

res6: Long = 30283


We can see that there are 283 columns left and 307511 rows left

Now we can try to drop the null rows and see what we've got:

In [20]:
val df6=df5.na.drop()
df6.count()

df6: org.apache.spark.sql.DataFrame = [EXT_SOURCE_1: string, EXT_SOURCE_2: string ... 359 more fields]
res7: Long = 87


In [1]:
val creditRiskdf = df6

Intitializing Scala interpreter ...

Spark Web UI available at http://82a2456298de:4043
SparkContext available as 'sc' (version = 2.4.3, master = local[*], app id = local-1558910889672)
SparkSession available as 'spark'


2019-05-26 22:48:05,878 WARN  [main] util.NativeCodeLoader (NativeCodeLoader.java:<clinit>(62)) - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2019-05-26 22:48:09,194 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2019-05-26 22:48:09,206 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
2019-05-26 22:48:09,208 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


<console>: 24: error: not found: value df6

## flagging duplicate values:

In [ ]:
//because the data has already been transformed & normalised use the IDCol (ARRindex:360) to check dupes

In [21]:
df6.dropDuplicates("SK_ID_CURR").count()

res8: Long = 87


We can see that there are no duplicate rows in this dataframe. 
We end up with 183875 rows and 283 columns.

Now write our dataframe in a csv file:

In [ ]:
df6.write.option("header", "true").csv("m_train_small_5.csv")

## displaying outliers:

In [22]:
//write function to check if there are any outliers in a given domain and show boxplot [Sean]
//slice a small test list to ensure functionality
//var test_lis = creditRiskFeatures.slice(0,2) //TEST var for codeblock
for (feature <- creditRiskFeatures) {
    //summarise column and query the DataFrame output
    var firstQ = creditRiskdf.select(feature).summary().where($"summary" === "25%").select(feature).first().mkString.toFloat
    var thirdQ = creditRiskdf.select(feature).summary().where($"summary" === "75%").select(feature).first().mkString.toFloat
    //use formulas below to test threshold of outliers
    var testValHigh = thirdQ + (1.5 * thirdQ - firstQ)
    var testValLow = firstQ - (1.5 * thirdQ - firstQ)
    //check to see if thresholds are exceeded in the column and count 
    var outHigh = creditRiskdf.filter(col(feature) > lit(testValHigh)).count()
    var outLow = creditRiskdf.filter(col(feature) < lit(testValLow)).count()
    
    //notify us whether or not column contains outliers
    if (outHigh > 0 || outLow > 0 ){
        println(feature.concat(": Contains outliers, #").concat((outHigh + outLow).toString))
    }else if (outHigh = 0 || outLow = 0){
        println(feature.concat(": Does not contain outliers"))
    }
}

<console>: 80: error: type mismatch;

In [23]:
// show boxplot [Tim]
def DoBoxes(feature: String): Int = {
        var testVal:Int = 0
val meandf = creditRiskdf.agg(feature -> "mean").first().mkString.toFloat                     //Tim 
//val stddevdf = creditRiskdf.agg(tfeature -> "stddev").first().mkString.toFloat                  // test Cell
val mindf = creditRiskdf.agg(feature -> "min").first().mkString.toFloat
val maxdf = creditRiskdf.agg(feature -> "max").first().mkString.toFloat
val q1 = maxdf * .25 
val q3 = maxdf * .75
//case class BoxEntry(key: String, grouping: String, Value: Float)                        // focus for week 3/4 to convert to diagrams
//val boxSeries = for(x <- 1 to 1) yield BoxEntry(tfeature,"sum",maxdf)
//val boxSeriesRDD = sc.parallelize(boxSeries)
println(feature+" "+mindf+" "+q1+" "+meandf+" "+q3+" "+maxdf)
// println(boxSeriesRDD)    
return testVal    
}

DoBoxes: (feature: String)Int


In [ ]:
//Initial call for the box plots.
for (feature <- creditRiskFeatures){
    DoBoxes(feature);                  // works but runs at about 14 secs per feature or column and will take 1.5 hours to complete
}

In [27]:
val feature = creditRiskFeatures(40)
DoBoxes(feature)

client_installments_NUM_INSTALMENT_VERSION_max_mean 0.0 2.4783949851989746 1.4393115 7.435184955596924 9.91358


feature: String = client_installments_NUM_INSTALMENT_VERSION_max_mean
res10: Int = 0


## reviewing data values for inconsistencies:

In [9]:
// define function to display summary statistics of specified feature

def incon (df:org.apache.spark.sql.DataFrame ,feature:org.apache.spark.sql.Column) {
    val f1 = df.select(feature)
    
    // data is in DataType String - convert to Float to faciltate 
    // computation of summary statistics
    val f2 = f1.withColumn(f1.columns.mkString, f1.col(
        f1.columns.mkString).cast(DataTypes.FloatType))
    f2.summary().show()
    val f3 = f2.na.drop()
    f3.sort(desc(f3.columns.mkString)).show(10)
    f3.sort(asc(f3.columns.mkString)).show(10)
}

incon: (df: org.apache.spark.sql.DataFrame, feature: org.apache.spark.sql.Column)Unit


In [29]:
// Randomly review 20 features for inconsistencies [Joel]

val r = scala.util.Random
var i:Int = 0
var randomlist = new Array[String](0)
while (i < 20){
    randomlist = randomlist :+ creditRiskFeatures(r.nextInt(creditRiskFeatures.length))
    i += 1}

for (i <- randomlist){
    incon(creditRiskdf,creditRiskdf(i))
}

+-------+---------------------------+
|summary|YEARS_BEGINEXPLUATATION_AVG|
+-------+---------------------------+
|  count|                      15392|
|   mean|         0.9776829908671644|
| stddev|        0.05945920180365278|
|    min|                        0.0|
|    25%|                     0.9767|
|    50%|                     0.9821|
|    75%|                     0.9866|
|    max|                        1.0|
+-------+---------------------------+

+---------------------------+
|YEARS_BEGINEXPLUATATION_AVG|
+---------------------------+
|                        1.0|
|                        1.0|
|                        1.0|
|                        1.0|
|                        1.0|
|                        1.0|
|                        1.0|
|                        1.0|
|                        1.0|
|                        1.0|
+---------------------------+
only showing top 10 rows

+---------------------------+
|YEARS_BEGINEXPLUATATION_AVG|
+---------------------------+
|      


+-----------------------------------+
|previous_loans_AMT_GOODS_PRICE_mean|
+-----------------------------------+
|                          3150000.0|
|                          3150000.0|
|                          2925000.0|
|                          2722971.5|
|                          2357573.2|
|                          2250000.0|
|                          2137500.0|
|                          2135250.0|
|                          2107672.0|
|                          2025000.0|
+-----------------------------------+
only showing top 10 rows

+-----------------------------------+
|previous_loans_AMT_GOODS_PRICE_mean|
+-----------------------------------+
|                                0.0|
|                                0.0|
|                                0.0|
|                             6876.0|
|                             8725.5|
|                             9855.0|
|                           10739.25|
|                            11227.5|
|                      


+-------------------------------------------------+
|client_cash_NAME_CONTRACT_STATUS_Signed_count_sum|
+-------------------------------------------------+
|                                              0.0|
|                                              0.0|
|                                              0.0|
|                                              0.0|
|                                              0.0|
|                                              0.0|
|                                              0.0|
|                                              0.0|
|                                              0.0|
|                                              0.0|
+-------------------------------------------------+
only showing top 10 rows

+-------+-------------------+
|summary|  NONLIVINGAREA_AVG|
+-------+-------------------+
|  count|              13395|
|   mean|0.02825275852033729|
| stddev|0.06957211282646082|
|    min|                0.0|
|    25%|                0.0|
|    


+---------------------------------------------------------+
|previous_loans_NAME_GOODS_CATEGORY_Audio/Video_count_norm|
+---------------------------------------------------------+
|                                                      0.0|
|                                                      0.0|
|                                                      0.0|
|                                                      0.0|
|                                                      0.0|
|                                                      0.0|
|                                                      0.0|
|                                                      0.0|
|                                                      0.0|
|                                                      0.0|
+---------------------------------------------------------+
only showing top 10 rows

+-------+-----------------------+
|summary|NONLIVINGAPARTMENTS_AVG|
+-------+-----------------------+
|  count|                   916


+-------+--------------------------------------------------+
|summary|previous_loans_NAME_PRODUCT_TYPE_x-sell_count_norm|
+-------+--------------------------------------------------+
|  count|                                             28700|
|   mean|                               0.20508649609982968|
| stddev|                               0.24479014535513974|
|    min|                                               0.0|
|    25%|                                               0.0|
|    50%|                                             0.125|
|    75%|                                        0.36363637|
|    max|                                               1.0|
+-------+--------------------------------------------------+

+--------------------------------------------------+
|previous_loans_NAME_PRODUCT_TYPE_x-sell_count_norm|
+--------------------------------------------------+
|                                               1.0|
|                                               1.0|
|

r: util.Random.type = scala.util.Random$@5e06f7a4
i: Int = 20
randomlist: Array[String] = Array(YEARS_BEGINEXPLUATATION_AVG, AMT_REQ_CREDIT_BUREAU_MON, bureau_DAYS_CREDIT_ENDDATE_sum, client_cash_NAME_CONTRACT_STATUS_Returned to the store_count_norm_max, FLAG_DOCUMENT_18, previous_loans_AMT_GOODS_PRICE_mean, bureau_AMT_CREDIT_SUM_max, previous_loans_NAME_CONTRACT_STATUS_Approved_count, previous_loans_CHANNEL_TYPE_Country-wide_count_norm, client_cash_NAME_CONTRACT_STATUS_Signed_count_sum, NONLIVINGAREA_AVG, bureau_DAYS_CREDIT_sum, previous_loans_NAME_PRODUCT_TYPE_x-sell_count_norm, previous_loans_AMT_APPLICATION_max, previous_loans_NAME_GOODS_CATEGORY_Audio/Video_count_norm, NONLIVINGAPARTMENTS_AVG, previous_loans_PRODUCT_COMBINATION_POS household with interest_count_norm, bureau_AMT_CRE...

In [ ]:
/* take a closer look at some features with potential inconsistencies
- previous_loans_DAYS_FIRST_DUE_mean
- previous_loans_SELLERPLACE_AREA_min
- previous_loans_DAYS_FIRST_DUE_sum
*/

In [13]:
// Investigate feature "previous_loans_DAYS_FIRST_DUE_mean"

val f1 = creditRiskdf.select("previous_loans_DAYS_FIRST_DUE_mean")
val f2 = f1.withColumn(f1.columns.mkString, f1.col(
    f1.columns.mkString).cast(DataTypes.FloatType))
// f2.summary().show()
val f3 = f2.na.drop()
// f3.sort(desc(f3.columns.mkString)).show(10)
// f3.sort(asc(f3.columns.mkString)).show(10)
f3.filter($"previous_loans_DAYS_FIRST_DUE_mean" === 365243).count()

f1: org.apache.spark.sql.DataFrame = [previous_loans_DAYS_FIRST_DUE_mean: string]
f2: org.apache.spark.sql.DataFrame = [previous_loans_DAYS_FIRST_DUE_mean: float]
f3: org.apache.spark.sql.DataFrame = [previous_loans_DAYS_FIRST_DUE_mean: float]
res6: Long = 52


In [31]:
// Investigate feature "previous_loans_SELLERPLACE_AREA_min"

val f1 = creditRiskdf.select("previous_loans_SELLERPLACE_AREA_min")
val f2 = f1.withColumn(f1.columns.mkString, f1.col(
    f1.columns.mkString).cast(DataTypes.FloatType))
// f2.summary().show()
val f3 = f2.na.drop()
// f3.sort(desc(f3.columns.mkString)).show(10)
// f3.sort(asc(f3.columns.mkString)).show(10)
f3.filter($"previous_loans_SELLERPLACE_AREA_min" < 0).count()

f1: org.apache.spark.sql.DataFrame = [previous_loans_SELLERPLACE_AREA_min: string]
f2: org.apache.spark.sql.DataFrame = [previous_loans_SELLERPLACE_AREA_min: float]
f3: org.apache.spark.sql.DataFrame = [previous_loans_SELLERPLACE_AREA_min: float]
res13: Long = 16070


Action: Values for this feature is inconsistent. "SELLERPLACE_AREA" should not be negative but this feature contains 163,911 lines of values < 0.  Will drop this feature from the dataset.

In [ ]:
// Investigate feature "previous_loans_DAYS_FIRST_DUE_sum"
// Reviewed output from earlier steps

Action: Similar to feature "previous_loans_DAYS_FIRST_DUE_mean". This has both +ve and -ve values when there should only be one convention. Will drop this feature from the dataset.

In [32]:
// Drop features with inconsistent values from dataset.

val incon_droplist = List("previous_loans_DAYS_FIRST_DUE_mean",
                      "previous_loans_SELLERPLACE_AREA_min",
                      "previous_loans_DAYS_FIRST_DUE_sum")

val df_lessInconData = creditRiskdf.drop(incon_droplist:_*)

incon_droplist: List[String] = List(previous_loans_DAYS_FIRST_DUE_mean, previous_loans_SELLERPLACE_AREA_min, previous_loans_DAYS_FIRST_DUE_sum)
df_lessInconData: org.apache.spark.sql.DataFrame = [EXT_SOURCE_1: string, EXT_SOURCE_2: string ... 356 more fields]


## data formatting:

In [ ]:
//iterate through features and apply the describe method and append to new dataframe
//secondary to this iterate through the values in the features and get datatypes, sql query this output for unique values

In [33]:
// find out if there are data in date formats [Joel]
// filter column headers containing 'date' for further investigation
val matcher = ".+date.+"
val dates = creditRiskFeatures.filter(x => x.toLowerCase matches matcher)

matcher: String = .+date.+
dates: Array[String] = Array(bureau_DAYS_CREDIT_ENDDATE_max, bureau_DAYS_ENDDATE_FACT_max, bureau_DAYS_CREDIT_UPDATE_max, bureau_DAYS_CREDIT_ENDDATE_mean, bureau_DAYS_ENDDATE_FACT_min, bureau_DAYS_CREDIT_ENDDATE_min, bureau_DAYS_CREDIT_ENDDATE_sum, bureau_DAYS_CREDIT_UPDATE_min, bureau_DAYS_ENDDATE_FACT_mean, bureau_DAYS_CREDIT_UPDATE_mean)


In [34]:
// review data format of columnbs with headers containing 'date' [Joel]
for (dateformat <- dates){
creditRiskdf.select(dateformat).show(10)
}

+------------------------------+
|bureau_DAYS_CREDIT_ENDDATE_max|
+------------------------------+
|                          null|
|                         471.0|
|                          null|
|                          null|
|                        1379.0|
|                       31087.0|
|                         638.0|
|                        -444.0|
|                         915.0|
|                        1818.0|
+------------------------------+
only showing top 10 rows

+----------------------------+
|bureau_DAYS_ENDDATE_FACT_max|
+----------------------------+
|                        null|
|                      -790.0|
|                        null|
|                        null|
|                      -123.0|
|                      -172.0|
|                      -102.0|
|                      -715.0|
|                      -926.0|
|                        null|
+----------------------------+
only showing top 10 rows

+-----------------------------+
|bureau_DAYS_CREDIT_

## Encoding categorical variables:

In [ ]:
//this has already been done for this dataset. No additional work required.